# boudary preserverance 
1. find border points
2. first calculate the DBP perserving for normal data augmentation
3. then change parametric umap, again calculate DBP perserving for train_data+augmentation_data+DBP
4. knn preserving evaluation

## load data

In [1]:
import torch
import math

import numpy as np
import torchvision
import torchvision.transforms as transforms
# from torch.utils.tensorboard import SummaryWriter
from tensorboardX import SummaryWriter
from cifar10_models import *

In [2]:
INPUT_SIZE = 2048
# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')
print(device)

cpu


In [14]:
# load model
model = resnet50(pretrained=True)
model.eval()
model.to(device)
print("Load Model successfully...")

Load Model successfully...


In [3]:
# hyperparameters
data_shape = (2048,)

ADD_NOISE = False
n_hidden = 500
dim_img = INPUT_SIZE  # number of pixels for a MNIST image
dim_z = 2

# train
n_epochs = 500
batch_size = 200
learn_rate = 0.001

### train_data

In [5]:
# CIFAR10 Test dataset and dataloader declaration
CIFAR_NORM = ((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize(*CIFAR_NORM)])

trainset = torchvision.datasets.CIFAR10(root='data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=200,
                                          shuffle=True, num_workers=2)
training_data = np.zeros((50000, 3, 32, 32))

for i, (data, target) in enumerate(trainloader, 0):
    r1, r2 = i * 200, (i + 1) * 200
    training_data[r1:r2] = data

train_pred_labels = np.zeros(50000)
raw_input_X = torch.from_numpy(training_data).to(device, dtype=torch.float)
input_X = np.zeros([len(raw_input_X), data_shape[0]])
output_Y = np.zeros(len(raw_input_X))
n_batches = max(math.ceil(len(raw_input_X) / batch_size), 1)
for b in range(n_batches):
    r1, r2 = b * batch_size, (b + 1) * batch_size
    inputs = raw_input_X[r1:r2]
    with torch.no_grad():
        pred = model.gap(inputs).cpu().numpy()
         input_X[r1:r2] = pred
        pred = model(inputs).cpu().numpy().argmax(axis=1)
        output_Y[r1:r2] = pred
train_data = input_X    # (50000,2048)
train_pred_labels = output_Y

training_data.shape, train_data.shape, train_pred_labels.shape

Files already downloaded and verified


((50000, 3, 32, 32), (50000, 2048), (50000,))

In [4]:
train_data = np.load("train_data.npy")
train_pred_labels = np.load("train_pred_labels.npy")
train_data.shape, train_pred_labels.shape

((50000, 2048), (50000,))

### test_data

In [19]:
# CIFAR10 Test dataset and dataloader declaration
CIFAR_NORM = ((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize(*CIFAR_NORM)])
testset = torchvision.datasets.CIFAR10(root='data', train=False,
                                        download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=200,
                                          shuffle=True, num_workers=2)
testing_data = np.zeros((10000, 3, 32, 32))
for i, (data, target) in enumerate(testloader, 0):
    r1, r2 = i * 200, (i + 1) * 200
    testing_data[r1:r2] = data

raw_input_X = torch.from_numpy(testing_data).to(device, dtype=torch.float)
input_X = np.zeros([len(raw_input_X), data_shape[0]])
output_Y = np.zeros(len(raw_input_X))
n_batches = max(math.ceil(len(raw_input_X) / batch_size), 1)
for b in range(n_batches):
    r1, r2 = b * batch_size, (b + 1) * batch_size
    inputs = raw_input_X[r1:r2]
    with torch.no_grad():
        pred = model.gap(inputs).cpu().numpy()
        input_X[r1:r2] = pred
        pred = model(inputs).cpu().numpy().argmax(axis=1)
        output_Y[r1:r2] = pred
test_data = input_X    # (10000,2048)
test_pred_labels = output_Y

testing_data.shape, test_data.shape, test_pred_labels.shape

Files already downloaded and verified


((10000, 3, 32, 32), (10000, 2048), (10000,))

In [5]:
test_data = np.load("test_data.npy")
test_pred_labels = np.load("test_pred_labels.npy")
test_data.shape, test_pred_labels.shape

((10000, 2048), (10000,))

### augmentation_data

In [6]:
import torch.nn as nn

criterion = nn.CrossEntropyLoss()
softmax = torch.nn.Softmax(dim=-1)
epsilons = [.01,.03,.05,.1]
epsilon = .01

In [7]:
def adv_attack(image, epsilon, data_grad):
    
    # Collect the element-wise sign of the data gradient
    sign_data_grad = torch.sign(data_grad)
    
    perturbed_image = image + epsilon*sign_data_grad
 
    # Adding clipping to maintain [0,1] range
    perturbed_image = torch.clamp(perturbed_image, 0, 1)

    return perturbed_image

In [8]:
# fgsm
print("attack with epsilon {}...".format(epsilon))

adv_data = np.zeros((50000, 3, 32, 32))
for i, (data, target) in enumerate(trainloader, 0):
    r1, r2 = i * 200, (i + 1) * 200
    data, target = data.to(device), target.to(device)

    # Set requires_grad attribute of tensor. Important for Attack
    data.requires_grad = True

    # Forward pass the data through the model
    output = model(data)
    init_pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability

    loss = criterion(output, target) # loss for ground-truth class

    # Back propogation
    model.zero_grad()
    loss.backward()

    # Collect data_grad
    data_grad = data.grad.data

    # Call Attack
    perturbed_data = adv_attack(data, epsilon, data_grad)

    adv_ex = perturbed_data.squeeze().detach().cpu().numpy()
    adv_data[r1:r2] = adv_ex

adv_data.shape

attack with epsilon 0.01...


(50000, 3, 32, 32)

In [10]:
raw_input_X = torch.from_numpy(adv_data).to(device, dtype=torch.float)
input_X = np.zeros([len(raw_input_X), data_shape[0]])
n_batches = max(math.ceil(len(raw_input_X) / batch_size), 1)
for b in range(n_batches):
    r1, r2 = b * batch_size, (b + 1) * batch_size
    inputs = raw_input_X[r1:r2]
    with torch.no_grad():
        pred = model.gap(inputs).cpu().numpy()
        input_X[r1:r2] = pred
augmentation_data = input_X    # (50000,2048)

adv_data.shape,augmentation_data.shape

((50000, 3, 32, 32), (50000, 2048))

In [ ]:
np.save("augmentation_data.npy", augmentation_data)

In [3]:
augmentation_data = np.load("augmentation_data.npy")

### border_data
here we define the border of resnet50 as the logits difference of top1 and top2 is less than 1.5
$$logit_{top1}-logit_{top2}<1.5$$

In [ ]:
# CIFAR10 Test dataset and dataloader declaration
CIFAR_NORM = ((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize(*CIFAR_NORM)])

trainset = torchvision.datasets.CIFAR10(root='data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=1,
                                          shuffle=True, num_workers=2)

In [ ]:
print("attack with epsilon {}...".format(epsilon))

border_data = np.zeros((50000, 3, 32, 32))
border_pred_labels = np.zeros(50000)
r=0
for i, (data, target) in enumerate(trainloader, 0):
    
    data, target = data.to(device), target.to(device)

    # Set requires_grad attribute of tensor. Important for Attack
    data.requires_grad = True
    j=1
    while True:
        output = model(data)
        if j ==1:
            print(output.detach().cpu().numpy())
        loss = criterion(output, target)  # loss for ground-truth class
        model.zero_grad()
        loss.backward()
        data_grad = data.grad.data

        perturbed_data = adv_attack(data, epsilon, data_grad)

        output = model(perturbed_data)
        sort, _ = torch.sort(output, dim=-1, descending=True)
        abs_dis = sort[0,0]-sort[0,1]
        print(abs_dis)
#         probabilities = softmax(output).detach().cpu().numpy()
#         entropy = -(probabilities*np.log(probabilities)).sum()/np.log(10)
        final_pred = output.max(1, keepdim=True)[1]

        adv_ex = perturbed_data.squeeze().detach().cpu().numpy()
        data = torch.from_numpy(np.expand_dims(adv_ex, axis=0)).to(device)
        data.requires_grad = True
        j = j+1
        if final_pred.item() != target:
            if abs_dis< 0.5:
                border_data[r] = adv_ex
                border_pred_labels[r] = final_pred.item()
                r = r+1
    #             print(i,j,target.detach().cpu().numpy(),"->",final_pred.item(),entropy)
    #             print(output.detach().cpu().numpy())
    #             print(probabilities)
            break
        if abs_dis < 1.5:
            border_data[r] = adv_ex
            border_pred_labels[r] = final_pred.item()
            r = r+1
            break
            
border_data.shape, border_pred_labels.shape, r

In [ ]:
raw_input_X = torch.from_numpy(border_data[:r]).to(device, dtype=torch.float)
input_X = np.zeros([len(raw_input_X), data_shape[0]])
n_batches = max(math.ceil(len(raw_input_X) / batch_size), 1)
for b in range(n_batches):
    r1, r2 = b * batch_size, (b + 1) * batch_size
    inputs = raw_input_X[r1:r2]
    with torch.no_grad():
        pred = model.gap(inputs).cpu().numpy()
        input_X[r1:r2] = pred
border_gap_data = input_X    # (50000,2048)

border_gap_data.shape

In [ ]:
border_pred_labels = border_pred_labels[:r]
border_gap_data.shape, border_pred_labels.shape

In [ ]:
np.save("ifgsm_augmentation_data.npy", border_gap_data)
np.save("adv_pred_labels.npy", border_pred_labels)

In [6]:
border_gap_data = np.load("ifgsm_augmentation_data.npy")
border_pred_labels = np.load("adv_pred_labels.npy")
border_gap_data.shape, border_pred_labels.shape

((18072, 2048), (18072,))

In [6]:
BP_bs = np.load("BP_bs.npy")
BP_bs_labels = np.load("BP_bs_labels.npy")
BP_bs.shape, BP_bs_labels.shape

((4497, 2048), (4497,))

#### find the N=5000 distance border points

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=5000).fit(border_gap_data)
kmeans.cluster_centers_.shape

In [10]:
# get labels
raw_input_X = torch.from_numpy(kmeans.cluster_centers_).to(device, dtype=torch.float)
output_Y = np.zeros(len(raw_input_X))
n_batches = max(math.ceil(len(raw_input_X) / batch_size), 1)
for b in range(n_batches):
    r1, r2 = b * batch_size, (b + 1) * batch_size
    inputs = raw_input_X[r1:r2]
    with torch.no_grad():
        pred = model.fc(inputs).cpu().numpy().argmax(axis=1)
        output_Y[r1:r2] = pred
border_center_labels = output_Y
border_center_labels.shape

(5000,)

In [11]:
np.save("border_center.npy",kmeans.cluster_centers_)
np.save("border_center_labels.npy", border_center_labels)

In [6]:
border_center = np.load("border_center.npy")
border_center_labels= np.load("border_center_labels.npy")
border_center.shape, border_center_labels.shape

((5000, 2048), (5000,))

## border preserving before DBP
1. train_data -> encoder_original
2. train_data,ifgsm_augmentation_data -> encoder
3. train_data, ifgsm_aumentation_data, DBP -> encoder_border
4. train_data, ifgsm_augmentation_data, DBP with modified parametric_umap_DBP -> encoder_DBP
5. train_data, ifgsm_aumentation_data, DBP negative_sampling_rate=10 -> encoder_border_10
6. train_data, DBP, with ratio 1:1:1 -> encoder_DBP_111

In [7]:
# define encoder
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [10]:
N=400

In [9]:
import os
save_location = "parametric_umap_models\parametric_umap_autoencoder"
# load model
# load encoder
encoder_output = os.path.join(save_location, "encoder_original")
if os.path.exists(encoder_output):
    load_encoder = tf.keras.models.load_model(encoder_output)
    print("Keras encoder model loaded from {}".format(encoder_output))

Keras encoder model loaded from parametric_umap_models\parametric_umap_autoencoder\encoder_DBP_010


In [10]:
low_center = load_encoder(border_center).cpu().numpy()
low_train = load_encoder(train_data).cpu().numpy()

NameError: name 'border_center' is not defined

In [9]:
## nndescent?
from sklearn.neighbors import KDTree
high_tree = KDTree(border_center) 
low_tree = KDTree(low_center)

In [11]:
_, high_ind = high_tree.query(train_data, k=N) 
_, low_ind = low_tree.query(low_train, k=N) 

In [12]:
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))

In [12]:
import pandas as pd
border_preserving = pd.DataFrame(columns = ['method','mean', 'max', 'min', 'median'])
border_preserving.loc[len(border_preserving)] = ["1",border_pres.mean(),border_pres.max(),border_pres.min(),np.median(border_pres)]

border_preserving

,method,mean,max,min,median
0,1,33.54342,75.0,0.0,33.0


In [13]:
border_pres.mean(),border_pres.max(),border_pres.min(),np.median(border_pres)

(235.8039, 414.0, 36.0, 218.0)

In [15]:
from sklearn.manifold import trustworthiness
tp_trustworthiness = trustworthiness(train_data, low_train, n_neighbors=10, metric='euclidean')
inter_dbp_trustworthiness = trustworthiness(border_center, low_center, n_neighbors=10, metric='euclidean')

In [16]:
inter_dbp_knn = pd.DataFrame(columns = ['method','trustworthiness'])
inter_dbp_knn.loc[len(inter_dbp_knn)] = ['1', inter_dbp_trustworthiness]
inter_dbp_knn

,method,trustworthiness
0,1,0.783059


In [17]:
tp_knn = pd.DataFrame(columns = ['method','trustworthiness'])
tp_knn.loc[len(tp_knn)] = ['1', tp_trustworthiness]

tp_knn

,method,trustworthiness
0,1,0.984133


In [14]:
# loader encoder and decoder
# import os
save_location = "parametric_umap_models\parametric_umap_autoencoder"
# load model
# load encoder
encoder_output = os.path.join(save_location, "encoder")
if os.path.exists(encoder_output):
    load_encoder = tf.keras.models.load_model(encoder_output)
    print("Keras encoder model loaded from {}".format(encoder_output))

# # load decoder
# decoder_output = os.path.join(save_location, "decoder_border")
# if os.path.exists(decoder_output):
#     load_decoder = tf.keras.models.load_model(decoder_output)
#     print("Keras decoder model loaded from {}".format(decoder_output))

Keras encoder model loaded from parametric_umap_models\parametric_umap_autoencoder\encoder


In [15]:
low_center = load_encoder(border_center).cpu().numpy()
low_train = load_encoder(train_data).cpu().numpy()

In [16]:
## nndescent?
# from sklearn.neighbors import KDTree
high_tree = KDTree(border_center) 
low_tree = KDTree(low_center)

In [17]:
_, high_ind = high_tree.query(train_data, k=N) 
_, low_ind = low_tree.query(low_train, k=N) 

In [18]:
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))

In [19]:
border_pres.mean(),border_pres.max(),border_pres.min(),np.median(border_pres)

(180.48812, 412.0, 9.0, 184.0)

In [23]:
tp_trustworthiness = trustworthiness(train_data, low_train, n_neighbors=10, metric='euclidean')
inter_dbp_trustworthiness = trustworthiness(border_center, low_center, n_neighbors=10, metric='euclidean')

In [24]:
border_preserving.loc[len(border_preserving)] = ["2",border_pres.mean(),border_pres.max(),border_pres.min(),np.median(border_pres)]

border_preserving

,method,mean,max,min,median
0,1,33.54342,75.0,0.0,33.0
1,2,18.86584,92.0,0.0,17.0


In [25]:
tp_knn.loc[len(tp_knn)] = ['2', tp_trustworthiness]

tp_knn

,method,trustworthiness
0,1,0.984133
1,2,0.983562


In [26]:
inter_dbp_knn.loc[len(inter_dbp_knn)] = ['2', inter_dbp_trustworthiness]

inter_dbp_knn

,method,trustworthiness
0,1,0.783059
1,2,0.830742


In [20]:
# loader encoder and decoder
# import os
save_location = "parametric_umap_models\parametric_umap_autoencoder"
# load model
# load encoder
encoder_output = os.path.join(save_location, "encoder_border")
if os.path.exists(encoder_output):
    load_encoder = tf.keras.models.load_model(encoder_output)
    print("Keras encoder model loaded from {}".format(encoder_output))

# # load decoder
# decoder_output = os.path.join(save_location, "decoder_DBP")
# if os.path.exists(decoder_output):
#     load_decoder = tf.keras.models.load_model(decoder_output)
#     print("Keras decoder model loaded from {}".format(decoder_output))

Keras encoder model loaded from parametric_umap_models\parametric_umap_autoencoder\encoder_border


In [21]:
low_center = load_encoder(border_center).cpu().numpy()
low_train = load_encoder(train_data).cpu().numpy()

In [22]:
## nndescent?
# from sklearn.neighbors import KDTree
high_tree = KDTree(border_center) 
low_tree = KDTree(low_center)

In [23]:
_, high_ind = high_tree.query(train_data, k=N) 
_, low_ind = low_tree.query(low_train, k=N)

In [24]:
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))

In [25]:
border_pres.mean(),border_pres.max(),border_pres.min(),np.median(border_pres)

(177.40916, 449.0, 0.0, 175.0)

In [32]:
tp_trustworthiness = trustworthiness(train_data, low_train, n_neighbors=10, metric='euclidean')
inter_dbp_trustworthiness = trustworthiness(border_center, low_center, n_neighbors=10, metric='euclidean')

In [33]:
border_preserving.loc[len(border_preserving)] = ["3",border_pres.mean(),border_pres.max(),border_pres.min(),np.median(border_pres)]

border_preserving

,method,mean,max,min,median
0,1,33.54342,75.0,0.0,33.0
1,2,18.86584,92.0,0.0,17.0
2,3,18.31906,93.0,0.0,11.0


In [34]:
tp_knn.loc[len(tp_knn)] = ['3', tp_trustworthiness]

tp_knn

,method,trustworthiness
0,1,0.984133
1,2,0.983562
2,3,0.983780


In [35]:
inter_dbp_knn.loc[len(inter_dbp_knn)] = ['3', inter_dbp_trustworthiness]

inter_dbp_knn

,method,trustworthiness
0,1,0.783059
1,2,0.830742
2,3,0.853619


In [26]:
# import os
save_location = "parametric_umap_models\parametric_umap_autoencoder"
# load model
# load encoder
encoder_output = os.path.join(save_location, "encoder_DBP")
if os.path.exists(encoder_output):
    load_encoder = tf.keras.models.load_model(encoder_output)
    print("Keras encoder model loaded from {}".format(encoder_output))

# # load decoder
# decoder_output = os.path.join(save_location, "decoder_DBP")
# if os.path.exists(decoder_output):
#     load_decoder = tf.keras.models.load_model(decoder_output)
#     print("Keras decoder model loaded from {}".format(decoder_output))

Keras encoder model loaded from parametric_umap_models\parametric_umap_autoencoder\encoder_DBP


In [27]:
low_center = load_encoder(border_center).cpu().numpy()
low_train = load_encoder(train_data).cpu().numpy()

In [28]:
## nndescent?
# from sklearn.neighbors import KDTree
high_tree = KDTree(border_center) 
low_tree = KDTree(low_center)

In [29]:
_, high_ind = high_tree.query(train_data, k=N) 
_, low_ind = low_tree.query(low_train, k=N)

In [30]:
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))

In [31]:
border_pres.mean(),border_pres.max(),border_pres.min(),np.median(border_pres)

(221.1106, 444.0, 21.0, 198.0)

In [41]:
tp_trustworthiness = trustworthiness(train_data, low_train, n_neighbors=10, metric='euclidean')
inter_dbp_trustworthiness = trustworthiness(border_center, low_center, n_neighbors=10, metric='euclidean')

In [42]:
tp_knn.loc[len(tp_knn)] = ['4', tp_trustworthiness]

tp_knn

,method,trustworthiness
0,1,0.984133
1,2,0.983562
2,3,0.983780
3,4,0.982697


In [43]:
inter_dbp_knn.loc[len(inter_dbp_knn)] = ['4', inter_dbp_trustworthiness]

inter_dbp_knn

,method,trustworthiness
0,1,0.783059
1,2,0.830742
2,3,0.853619
3,4,0.828469


In [44]:
border_preserving.loc[len(border_preserving)] = ["4",border_pres.mean(),border_pres.max(),border_pres.min(),np.median(border_pres)]
border_preserving

,method,mean,max,min,median
0,1,33.54342,75.0,0.0,33.0
1,2,18.86584,92.0,0.0,17.0
2,3,18.31906,93.0,0.0,11.0
3,4,26.02318,83.0,0.0,16.0


In [32]:
import os
save_location = "parametric_umap_models\parametric_umap_autoencoder"
# load model
# load encoder
encoder_output = os.path.join(save_location, "encoder_border_10")
if os.path.exists(encoder_output):
    load_encoder = tf.keras.models.load_model(encoder_output)
    print("Keras encoder model loaded from {}".format(encoder_output))

# # load decoder
# decoder_output = os.path.join(save_location, "decoder_DBP")
# if os.path.exists(decoder_output):
#     load_decoder = tf.keras.models.load_model(decoder_output)
#     print("Keras decoder model loaded from {}".format(decoder_output))

Keras encoder model loaded from parametric_umap_models\parametric_umap_autoencoder\encoder_border_10


In [33]:
low_center = load_encoder(border_center).cpu().numpy()
low_train = load_encoder(train_data).cpu().numpy()

In [34]:
## nndescent?
from sklearn.neighbors import KDTree
high_tree = KDTree(border_center) 
low_tree = KDTree(low_center)

In [35]:
_, high_ind = high_tree.query(train_data, k=100) 
_, low_ind = low_tree.query(low_train, k=100)

In [36]:
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))

In [37]:
border_pres.mean(),border_pres.max(),border_pres.min(),np.median(border_pres)

(14.79106, 97.0, 0.0, 13.0)

In [50]:
from sklearn.manifold import trustworthiness
tp_trustworthiness = trustworthiness(train_data, low_train, n_neighbors=10, metric='euclidean')
inter_dbp_trustworthiness = trustworthiness(border_center, low_center, n_neighbors=10, metric='euclidean')

In [51]:
border_preserving = pd.DataFrame(columns = ['method','mean', 'max', 'min', 'median'])
border_preserving.loc[len(border_preserving)] = ["5",border_pres.mean(),border_pres.max(),border_pres.min(),np.median(border_pres)]

border_preserving

,method,mean,max,min,median
0,5,14.79106,97.0,0.0,13.0


In [52]:
tp_knn = pd.DataFrame(columns = ['method','trustworthiness'])
tp_knn.loc[len(tp_knn)] = ['5', tp_trustworthiness]

tp_knn

,method,trustworthiness
0,5,0.983048


In [53]:
inter_dbp_knn = pd.DataFrame(columns = ['method','trustworthiness'])
inter_dbp_knn.loc[len(inter_dbp_knn)] = ['5', inter_dbp_trustworthiness]

inter_dbp_knn

,method,trustworthiness
0,5,0.840593


In [54]:
tp_trustworthiness = trustworthiness(train_data, low_train, n_neighbors=30, metric='euclidean')
inter_dbp_trustworthiness = trustworthiness(border_center, low_center, n_neighbors=30, metric='euclidean')

In [55]:
tp_knn.loc[len(tp_knn)] = ['5_30', tp_trustworthiness]
inter_dbp_knn.loc[len(inter_dbp_knn)] = ['5_30', inter_dbp_trustworthiness]

In [56]:
tp_trustworthiness = trustworthiness(train_data, low_train, n_neighbors=50, metric='euclidean')
inter_dbp_trustworthiness = trustworthiness(border_center, low_center, n_neighbors=50, metric='euclidean')
tp_knn.loc[len(tp_knn)] = ['5_50', tp_trustworthiness]
inter_dbp_knn.loc[len(inter_dbp_knn)] = ['5_50', inter_dbp_trustworthiness]

In [57]:
tp_trustworthiness = trustworthiness(train_data, low_train, n_neighbors=100, metric='euclidean')
inter_dbp_trustworthiness = trustworthiness(border_center, low_center, n_neighbors=100, metric='euclidean')
tp_knn.loc[len(tp_knn)] = ['5_100', tp_trustworthiness]
inter_dbp_knn.loc[len(inter_dbp_knn)] = ['5_100', inter_dbp_trustworthiness]

In [58]:
tp_knn

,method,trustworthiness
0,5,0.983048
1,5_30,0.982877
2,5_50,1.017189
3,5_100,0.999946


In [59]:
inter_dbp_knn

,method,trustworthiness
0,5,0.840593
1,5_30,0.838659
2,5_50,0.835383
3,5_100,0.828794


In [8]:
import os
save_location = "parametric_umap_models\parametric_umap_autoencoder"
# load model
# load encoder
encoder_output = os.path.join(save_location, "encoder_DBP_111")
if os.path.exists(encoder_output):
    load_encoder = tf.keras.models.load_model(encoder_output)
    print("Keras encoder model loaded from {}".format(encoder_output))

Keras encoder model loaded from parametric_umap_models\parametric_umap_autoencoder\encoder_DBP_111


In [9]:
low_center = load_encoder(border_center).cpu().numpy()
low_train = load_encoder(train_data).cpu().numpy()

Instructions for updating:
Use tf.identity instead.


In [10]:
## nndescent?
from sklearn.neighbors import KDTree
high_tree = KDTree(border_center) 
low_tree = KDTree(low_center)

In [11]:
_, high_ind = high_tree.query(train_data, k=100) 
_, low_ind = low_tree.query(low_train, k=100)

In [12]:
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))

In [13]:
from sklearn.manifold import trustworthiness
tp_trustworthiness = trustworthiness(train_data, low_train, n_neighbors=10, metric='euclidean')
inter_dbp_trustworthiness = trustworthiness(border_center, low_center, n_neighbors=10, metric='euclidean')

In [14]:
tp_trustworthiness, inter_dbp_trustworthiness

(0.9805236570536866, 0.8362489798374962)

In [15]:
border_pres.mean(),border_pres.max(),border_pres.min(),np.median(border_pres)

(23.88582, 87.0, 0.0, 19.0)

In [16]:
_, high_ind = high_tree.query(train_data, k=500) 
_, low_ind = low_tree.query(low_train, k=500)

In [17]:
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))

In [18]:
border_pres.mean(),border_pres.max(),border_pres.min(),np.median(border_pres)

(210.44836, 401.0, 14.0, 221.0)

In [9]:
import os
save_location = "parametric_umap_models\parametric_umap_autoencoder"
# load model
# load encoder
encoder_output = os.path.join(save_location, "encoder_DBP_122")
if os.path.exists(encoder_output):
    load_encoder = tf.keras.models.load_model(encoder_output)
    print("Keras encoder model loaded from {}".format(encoder_output))

Keras encoder model loaded from parametric_umap_models\parametric_umap_autoencoder\encoder_DBP_122


In [10]:
low_center = load_encoder(border_center).cpu().numpy()
low_train = load_encoder(train_data).cpu().numpy()

Instructions for updating:
Use tf.identity instead.


In [11]:
## nndescent?
from sklearn.neighbors import KDTree
high_tree = KDTree(border_center) 
low_tree = KDTree(low_center)

In [12]:
_, high_ind = high_tree.query(train_data, k=N) 
_, low_ind = low_tree.query(low_train, k=N)

In [14]:
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))

In [15]:
border_pres.mean(),border_pres.max(),border_pres.min(),np.median(border_pres)

(213.3516, 399.0, 5.0, 206.0)

# test encoder/decoder
| |M_tptp(original)|	M_tptp(core)|	M_tpdp(core)|	M_(new complex)(core)|
|---|---|---|---|---|
|K=10|training edge set||||	|	
||testing edge set||||		
|K=20|training edge set	||||			
||testing edge set||||				
|K=30|training edge set	||||		
||testing edge set||||				
|K=50|training edge set|||||
||testing edge set||||			

In [26]:
import os
save_location = "parametric_umap_models\dbp_exp"
# load model
# load encoder
encoder_output = os.path.join(save_location, "encoder_core_tpdp")
if os.path.exists(encoder_output):
    load_encoder = tf.keras.models.load_model(encoder_output)
    print("Keras encoder model loaded from {}".format(encoder_output))

Keras encoder model loaded from parametric_umap_models\dbp_exp\encoder_core_tpdp


In [27]:
low_center = load_encoder(border_center).cpu().numpy()
# low_center = load_encoder(BP_bs).cpu().numpy()
low_train = load_encoder(train_data).cpu().numpy()

In [28]:
## nndescent?
from sklearn.neighbors import KDTree
high_tree = KDTree(border_center) 
# high_tree = KDTree(BP_bs) 
low_tree = KDTree(low_center)

In [11]:
_, high_ind = high_tree.query(train_data, k=10) 
_, low_ind = low_tree.query(low_train, k=10)
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=10
border_pres.mean(), border_pres.max(),border_pres.min()

(1.42742, 8.0, 0.0)

In [12]:
_, high_ind = high_tree.query(train_data, k=20) 
_, low_ind = low_tree.query(low_train, k=20)
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=20
border_pres.mean(), border_pres.max(),border_pres.min()

(3.94338, 17.0, 0.0)

In [13]:
_, high_ind = high_tree.query(train_data, k=30) 
_, low_ind = low_tree.query(low_train, k=30)
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=30
border_pres.mean(), border_pres.max(),border_pres.min()

(7.5892, 22.0, 0.0)

In [14]:
_, high_ind = high_tree.query(train_data, k=50) 
_, low_ind = low_tree.query(low_train, k=50)
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=50
border_pres.mean(), border_pres.max(),border_pres.min()

(14.10516, 35.0, 0.0)

In [18]:
_, high_ind = high_tree.query(train_data, k=10) 
_, low_ind = low_tree.query(low_train, k=10)
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp core K=10
border_pres.mean(), border_pres.max(),border_pres.min()

(1.44746, 9.0, 0.0)

In [19]:
_, high_ind = high_tree.query(train_data, k=20) 
_, low_ind = low_tree.query(low_train, k=20)
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp core K=20
border_pres.mean(), border_pres.max(),border_pres.min()

(4.5924, 16.0, 0.0)

In [20]:
_, high_ind = high_tree.query(train_data, k=30) 
_, low_ind = low_tree.query(low_train, k=30)
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp core K=30
border_pres.mean(), border_pres.max(),border_pres.min()

(8.17968, 22.0, 0.0)

In [21]:
_, high_ind = high_tree.query(train_data, k=50) 
_, low_ind = low_tree.query(low_train, k=50)
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp core K=50
border_pres.mean(), border_pres.max(),border_pres.min()

(15.64786, 37.0, 0.0)

In [25]:
_, high_ind = high_tree.query(train_data, k=10) 
_, low_ind = low_tree.query(low_train, k=10)
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp core rho=0 K=10
border_pres.mean(), border_pres.max(),border_pres.min()

(1.39768, 9.0, 0.0)

In [29]:
_, high_ind = high_tree.query(train_data, k=20) 
_, low_ind = low_tree.query(low_train, k=20)
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp core rho=0 K=20
border_pres.mean(), border_pres.max(),border_pres.min()

(4.6141, 16.0, 0.0)

In [30]:
_, high_ind = high_tree.query(train_data, k=30) 
_, low_ind = low_tree.query(low_train, k=30)
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp core rho=0 K=30
border_pres.mean(), border_pres.max(),border_pres.min()

(8.24208, 24.0, 0.0)

In [31]:
_, high_ind = high_tree.query(train_data, k=50) 
_, low_ind = low_tree.query(low_train, k=50)
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp core rho=0 K=50
border_pres.mean(), border_pres.max(),border_pres.min()

(16.2586, 40.0, 0.0)

In [29]:
_, high_ind = high_tree.query(train_data, k=10) 
_, low_ind = low_tree.query(low_train, k=10)
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tpdbp core K=10
border_pres.mean(), border_pres.max(),border_pres.min()

(0.51554, 10.0, 0.0)

In [19]:
_, high_ind = high_tree.query(train_data, k=20) 
_, low_ind = low_tree.query(low_train, k=20)
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tpdbp core K=20
border_pres.mean(), border_pres.max(),border_pres.min()

(1.45436, 18.0, 0.0)

In [20]:
_, high_ind = high_tree.query(train_data, k=30) 
_, low_ind = low_tree.query(low_train, k=30)
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tpdbp core K=30
border_pres.mean(), border_pres.max(),border_pres.min()

(2.43646, 28.0, 0.0)

In [21]:
_, high_ind = high_tree.query(train_data, k=50) 
_, low_ind = low_tree.query(low_train, k=50)
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tpdbp core K=50
border_pres.mean(), border_pres.max(),border_pres.min()

(5.22802, 46.0, 0.0)

In [12]:
_, high_ind = high_tree.query(train_data, k=10) 
_, low_ind = low_tree.query(low_train, k=10)
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp tpdbp core K=10
border_pres.mean(), border_pres.max(),border_pres.min()

(1.31982, 9.0, 0.0)

In [15]:
_, high_ind = high_tree.query(train_data, k=20) 
_, low_ind = low_tree.query(low_train, k=20)
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp tpdbp core K=20
border_pres.mean(), border_pres.max(),border_pres.min()

(3.63902, 16.0, 0.0)

In [13]:
_, high_ind = high_tree.query(train_data, k=30) 
_, low_ind = low_tree.query(low_train, k=30)
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp tpdbp core K=30
border_pres.mean(), border_pres.max(),border_pres.min()

(6.4789, 24.0, 0.0)

In [14]:
_, high_ind = high_tree.query(train_data, k=50) 
_, low_ind = low_tree.query(low_train, k=50)
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp tpdbp core K=50
border_pres.mean(), border_pres.max(),border_pres.min()

(12.9191, 38.0, 0.0)

In [15]:
# k=30
_, high_ind = high_tree.query(train_data, k=10) 
_, low_ind = low_tree.query(low_train, k=10)
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# new complex core K=10
border_pres.mean(), border_pres.max(),border_pres.min()

(1.17478, 9.0, 0.0)

In [16]:
_, high_ind = high_tree.query(train_data, k=20) 
_, low_ind = low_tree.query(low_train, k=20)
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# new complex core K=20
border_pres.mean(), border_pres.max(),border_pres.min()

(6.33074, 19.0, 0.0)

In [17]:
_, high_ind = high_tree.query(train_data, k=30) 
_, low_ind = low_tree.query(low_train, k=30)
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# new complex core K=30
border_pres.mean(), border_pres.max(),border_pres.min()

(14.1065, 29.0, 0.0)

In [18]:
_, high_ind = high_tree.query(train_data, k=50) 
_, low_ind = low_tree.query(low_train, k=50)
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# new complex core K=10
border_pres.mean(), border_pres.max(),border_pres.min()

(26.6353, 46.0, 0.0)

In [22]:
# k=15
_, high_ind = high_tree.query(train_data, k=10) 
_, low_ind = low_tree.query(low_train, k=10)
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# new complex core K=10
border_pres.mean(), border_pres.max(),border_pres.min()

(2.8041, 10.0, 0.0)

In [23]:
# k=15
_, high_ind = high_tree.query(train_data, k=20) 
_, low_ind = low_tree.query(low_train, k=20)
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# new complex core K=20
border_pres.mean(), border_pres.max(),border_pres.min()

(7.8586, 20.0, 0.0)

In [24]:
# k=15
_, high_ind = high_tree.query(train_data, k=30) 
_, low_ind = low_tree.query(low_train, k=30)
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# new complex core K=30
border_pres.mean(), border_pres.max(),border_pres.min()

(12.90748, 28.0, 0.0)

In [25]:
# k=15
_, high_ind = high_tree.query(train_data, k=50) 
_, low_ind = low_tree.query(low_train, k=50)
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# new complex core K=50
border_pres.mean(), border_pres.max(),border_pres.min()

(22.87476, 46.0, 0.0)